In [34]:
import os 
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
data_dir = ''
os.chdir(data_dir)
os.getcwd()

''

In [35]:
def reshape_data(df, sheet,idx_col=0):
    '''一行四个氨基酸转换为一行表示一个氨基酸
    '''
    sdf = df.parse(sheet, header=None, index_col=idx_col)
    sdf = sdf.fillna(method='ffill')
    try:
        sdf.index = [idx.strip() for idx in list(sdf.index)]
    except:
        pass
    colnames = ['aa', 'codon', 'num', 'rscu']
    new_df = pd.DataFrame()
    for c in range(0, 16, 4):
        t_col = sdf.iloc[:, c:c + 4]
        t_col.columns = colnames
        new_df = new_df.append(t_col)
    new_df_rscu_mean = new_df.groupby(['aa', 'codon']).mean()
    return new_df_rscu_mean


def get_high_drscu(high, low):
    if list(high.index) != list(low.index):
        raise sysExit('error sb')
    res = pd.concat([high, low], axis=1)
    res['d_rscu'] = res['rscu_high'] - res['rscu_low']
    res = res.sort_values(by=['d_rscu'], ascending=False)
    res_high = res[res['d_rscu'] > 0.08]
    print(res_high.head())
    high_drscu_codons = [c[1] for c in list(res_high.index)]
    return set(high_drscu_codons)

def get_rscu1(tot_rscu):
    stot = tot_rscu[tot_rscu['rscu']>1]
    #print(stot.head)
    tot_high_rscu_codons = [c[1] for c in list(stot.index)]
    return set(tot_high_rscu_codons)

def compare_rscu(high_drscu_codons, low_rscu_codons):
    return high_drscu_codons & low_rscu_codons

def get_opt_codons(sp_rscu):
    df = pd.ExcelFile(sp_rscu)
    high = reshape_data(df, 1)
    high.columns = ['nums_high', 'rscu_high']
    low = reshape_data(df, 2)
    low.columns = ['nums_low','rscu_low']
    tot = reshape_data(df, 3, idx_col=None)
    high_rscu_codons = get_rscu1(tot)
    high_drscu_codons = get_high_drscu(high, low)
    opt_codons = compare_rscu(high_drscu_codons, high_rscu_codons)
    return opt_codons

In [36]:
sp_rscus = [
    '1lt.rscu.xlsx', '2ts.rscu.xlsx', '3ms.rscu.xlsx', '4mf.rscu.xlsx',
    '5mg.rscu.xlsx'
]

for sp_rscu in sp_rscus:
    opt_codons = get_opt_codons(sp_rscu)
    print(sp_rscu, opt_codons)
    
    

           nums_high  rscu_high  nums_low  rscu_low  d_rscu
aa  codon                                                  
Leu CUA          2.6      0.800       0.2     0.032   0.768
Ala GCA          1.4      1.066       0.6     0.306   0.760
Val GUA          3.4      1.604       2.8     0.868   0.736
Leu CUG          3.6      1.170       2.0     0.512   0.658
Thr ACG          1.6      1.440       3.8     0.784   0.656
1lt.rscu.xlsx {'CGU', 'GCC', 'AAA', 'GUU', 'GUA'}
           nums_high  rscu_high  nums_low  rscu_low    d_rscu
aa  codon                                                    
Leu CUA          3.6      1.086  0.200000  0.032000  1.054000
Val GUA          4.8      1.816  2.800000  0.860000  0.956000
Leu CUG          4.2      1.352  1.666667  0.426667  0.925333
Ala GCA          1.8      1.106  0.600000  0.306000  0.800000
Gly GGG          3.6      1.294  2.200000  0.516000  0.778000
2ts.rscu.xlsx {'CGU', 'GCC', 'AAA', 'GUU', 'GUA'}
           nums_high  rscu_high  nums_low  rsc

In [37]:
# rscu1 = '1lt.rscu.xlsx'
# df1 = pd.ExcelFile(rscu1)
# df1.sheet_names
# tot = reshape_data(df1, 3, None)
# tmp = tot.T
# tot.reset_index(inplace=True)
# tot.set_index(['aa'],inplace=True)
# tot.head()
# plt.figure(dpi=200,figsize=(15,10))
# tot['num'].plot(kind='bar')